In [1]:
import amazon_dataset
import movielens_dataset
import feature_extraction_text

import numpy as np

# For Amazon Dataset

In [9]:
dataset = 'Musical_Instruments'
items = amazon_dataset.items_df(dataset)
items.sample(n=3)


,asin,description,title,brand,main_cat,rank,price,image_slug,image_url,feature,category,tech_detail,item_id
id,,,,,,,,,,,,,
115884,B01CX858RG,,Tibetan Singing Bowl Set ~ Buddha Figure Desig...,Maha Bodhi,Musical Instruments,">#25,159 in Musical Instruments (See Top 100 i...",$7.99,[41MtLYQwgrL],[https://m.media-amazon.com/images/I/41MtLYQwg...,[HANDCRAFTED: Handcrafted and Hand Hammered ma...,"[Musical Instruments, Drums & Percussion, Hand...",None,B01CX858RG
88225,B00K87GKOA,The Snapz Acoustic Guitar Bridge Pin Puller us...,BIGROCK INNOVATIONS SNAPZ Acoustic Guitar Brid...,BigRock Innovations,Musical Instruments,">#26,512 in Musical Instruments (See Top 100 i...",$8.95,[41Phve6OcEL],[https://images-na.ssl-images-amazon.com/image...,[PATENT PENDING TECHNOLOGY: The Snapz Acoustic...,"[Musical Instruments, Instrument Accessories, ...",None,B00K87GKOA
674,B0000WS6QI,CLARITY AND COMFORT FOR THE MODERN DJ One plac...,Numark HF125 | On-Ear DJ Headphones,Numark,Musical Instruments,">#7,591 in Musical Instruments (See Top 100 in...",$1.96,[41n2TafLIUL],[https://m.media-amazon.com/images/I/41n2TafLI...,[Flexible 7-position independently adjustable ...,"[Musical Instruments, Electronic Music, DJ & K...",None,B0000WS6QI


In [ ]:
extracted_bert_features = feature_extraction_text.extract_bert_text_features(items)

In [10]:
np.savez_compressed(
    amazon_dataset.BASE_DATA_FOLDER / f'{dataset}_bert_features.npz',
    **extracted_bert_features
)

In [ ]:
extacted_clip_features = feature_extraction_text.extract_clip_text_features(
    items
)

In [12]:
np.savez_compressed(
    amazon_dataset.BASE_DATA_FOLDER / f'{dataset}_cliptext_features.npz',
    **extacted_clip_features
)

# For the Movielens dataset

In [5]:
dataset = 'ml-25m'
items = movielens_dataset.items_df(dataset)
items.sample(n=3)

,title,genres,year,original_title,title_imdb,genres_ml,year_imdb,imdb_id,writer,actor,...,runtimeMinutes,genres_imdb,directors,writers,synopsis,summary,image_url,description,item_id,image_slug
movie_id,,,,,,,,,,,,,,,,,,,,,
188441,The Mohican Comes Home,[Comedy],2016,NaN,The Mohican Comes Home,Comedy,2016,tt4691804,\N,nm3877188|nm0256628|nm0452114|nm4405816|nm0559430,...,125,Comedy,nm3255587,nm3255587,None,A punk comes back to his family home on a smal...,https://m.media-amazon.com/images/M/MV5BNGEwYj...,A punk comes back to his family home on a smal...,188441,[188441]
162428,Jane's Journey,[Documentary],2011,NaN,Jane's Journey,Documentary,2011,tt1326283,\N,\N,...,107,Biography|Documentary,nm0460564,nm0460564,None,"More than 20 years ago, Dr. Jane Goodall, now ...",https://m.media-amazon.com/images/M/MV5BNjIxMj...,"More than 20 years ago, Dr. Jane Goodall, now ...",162428,[162428]
70490,"Trail Beyond, The","[Action, Adventure, Western]",1934,NaN,"Trail Beyond, The",Action|Adventure|Western,1934,tt0025903,nm0193617|nm0663858,nm0000078|nm0001935|nm0000890,...,55,Action|Adventure|Drama,nm0102908,nm0193617|nm0663858,None,Rod Drew hunts for a missing girl and finds hi...,https://m.media-amazon.com/images/M/MV5BNTJjNj...,Rod Drew hunts for a missing girl and finds hi...,70490,[70490]


In [6]:
extracted_bert_features = feature_extraction_text.extract_bert_text_features(items)

Some weights of the model checkpoint at bert-large-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
  0%|          | 48.0/62.4k [00:00<18:51, 55.1item/s]/home/igui/src/VBPR/.venv/lib/python3.8/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarnin

In [7]:
np.savez_compressed(
    movielens_dataset.BASE_DATA_FOLDER / f'{dataset}_bert_features.npz',
    **extracted_bert_features
)

In [8]:
extacted_clip_features = feature_extraction_text.extract_clip_text_features(
    items
)

100%|██████████| 62.4k/62.4k [07:13<00:00, 144product/s]  


In [9]:
np.savez_compressed(
    movielens_dataset.BASE_DATA_FOLDER / f'{dataset}_cliptext_features.npz',
    **extacted_clip_features
)